<a href="https://colab.research.google.com/github/aleks-haksly/ClickHouse/blob/main/practice/part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
#!pip install clickhouse_driver --quiet
from clickhouse_driver import Client
from google.colab import userdata
import ipywidgets as widgets

In [17]:
# connection requisites
host = 'clickhouse.lab.karpov.courses'
dbase = 'default'
port = 9000
user = 'student'
password = userdata.get('ch2020[student]')

In [7]:
client = Client(host=host, port=port, user=user,
                password=password, database=dbase)

## [Airbnb](https://github.com/aleks-haksly/ClickHouse/blob/main/practice/README.md#airbnb) Data Base


### Задание 4.1
Посчитайте среднее расстояние до центра города и выведите идентификаторы объявлений о сдаче отдельных комнат, для которых расстояние оказалось меньше среднего. Результат отсортируйте по убыванию, тем самым выбрав комнату, которая является наиболее удаленной от центра, но при этом расположена ближе, чем остальные комнаты в среднем.

In [ ]:
sql = f"""
WITH avg_room_distance as (
SELECT
  avg(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude))) AS avg_distance
FROM
  listings
WHERE
  room_type = 'Private room')

SELECT
  id,
  geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS distance
FROM
  listings
WHERE
  room_type = 'Private room' and distance < (SELECT avg_distance FROM avg_room_distance)
ORDER BY distance DESC
LIMIT 1
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,id,distance
0,22306678,4608.981934


### Задание 5.2
Отберите 3 объявления из таблицы listings, которые:
* находятся на расстоянии от центра меньше среднего
*	обойдутся дешевле 100$ в день (price с учетом cleaning_fee, который добавляется к общей сумме за неделю, т.е его нужно делить на кол-во дней)
*	имеют последние отзывы (last_review), начиная с 1 сентября 2018 года
*	имеют WiFi в списке удобств (amenities) <br>
Отсортируйте полученные значения по убыванию review_scores_rating

In [ ]:
sql = f"""
WITH avg_room_distance as (
SELECT
  avg(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude))) AS avg_distance
FROM
  listings)

SELECT
  id,
  host_id,
  geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS distance,
  toFloat32OrNull(replaceRegexpAll(price, '[$,]', '')) + toFloat32OrNull(replaceRegexpAll(cleaning_fee, '[$,]', '')) / 7 AS price_cfee ,
  last_review,
  review_scores_rating,
  amenities
FROM
  listings
WHERE
  distance < (SELECT avg_distance FROM avg_room_distance) AND
  price_cfee < 100 AND
  toDateOrNull(last_review) >= toDate('2018-09-01') AND
  multiSearchAnyCaseInsensitive(amenities, ['wifi']) = 1
ORDER BY toFloat32OrNull(review_scores_rating)  DESC
LIMIT 3
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,id,host_id,distance,price_cfee,last_review,review_scores_rating,amenities
0,28071741,39740677,2528.391602,53.571429,2018-11-04,100.0,"{Wifi,Kitchen,Elevator,Heating,Washer,Dryer,""S..."
1,15186943,20173692,4208.450195,81.428571,2018-09-16,100.0,"{Wifi,Kitchen,Heating,Washer,Essentials,""Lapto..."
2,7197260,19577506,3922.759277,52.857143,2018-09-02,100.0,"{Wifi,Kitchen,""Pets live on this property"",""Ot..."


### Задание 5.3
Давайте найдем в таблице calendar_summary те доступные (available='t') объявления, у которых число отзывов от уникальных пользователей в таблице reviews выше среднего.
Для простоты будем считать, что отзыв — это уникальный посетитель на уникальное жилье, не учитывая возможные повторные отзывы от того же посетителя. Отсортируйте результат по возрастанию listing_id и покажите первый результат.

In [8]:
sql = f"""
WITH uniq_listings AS (
SELECT
  listing_id,
  uniqExact(reviewer_id) AS cnt
FROM
  reviews
GROUP BY
  listing_id
HAVING cnt > (SELECT uniqExact(reviewer_id) / uniqExact(listing_id) FROM reviews) )

SELECT * FROM calendar_summary cs
JOIN uniq_listings u on u.listing_id = cs.listing_id
WHERE available = 't'
ORDER BY cs.listing_id
limit 1
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,listing_id,date,available,price,u.listing_id,cnt
0,2015,2019-05-05,t,$76.00,2015,118


## [Mobile installs](https://github.com/aleks-haksly/ClickHouse/blob/main/practice/README.md#mobile-installs) Data Base

### Задание 5.4
Используйте таблицу **checks** и разделите всех покупателей на сегменты:
*	А — средний чек покупателя менее 5 ₽
*	B — средний чек покупателя от 5-10 ₽
*	C — средний чек покупателя от 10-20 ₽
*	D — средний чек покупателя от 20 ₽ <br>
и посчитайте, сколько клиентов приходится на каждый сегмент и сколько доходов он приносит.


In [10]:
sql = f"""
WITH user_segmets as (
  SELECT
    UserID,
    avg(Rub) as avg_check,
    (CASE
        WHEN avg_check < 5 THEN 'A'
        WHEN avg_check < 10 THEN 'B'
        WHEN avg_check < 20 THEN 'C'
        ELSE 'D'
      END) as segment
  FROM checks
  GROUP BY UserID)

SELECT
  us.segment AS segment,
  SUM(c.Rub) AS sum,
  uniqExact(UserID) AS uniq_users
FROM
  user_segmets us
JOIN checks c ON us.UserID = c.UserID
GROUP BY us.segment
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,segment,sum,uniq_users
0,B,72257465,991535
1,D,250551286,688705
2,C,223388252,999564
3,A,65057410,2494577


## [Airbnb](https://github.com/aleks-haksly/ClickHouse/blob/main/practice/README.md#airbnb) Data Base


### Задание 5.5
Предположим, что в выборе жилья нас интересует только два параметра: наличие кухни (kitchen) и гибкой системы отмены (flexible), причем первый в приоритете.
Создайте с помощью оператора CASE колонку с обозначением группы, в которую попадает жилье из таблицы listings:
1.	'good', если в удобствах (amenities) присутствует кухня и система отмены (cancellation_policy) гибкая
2.	'ok', если в удобствах есть кухня, но система отмены не гибкая
3.	'not ok' во всех остальных случаях. <br>
Результат отсортируйте по новой колонке по возрастанию, установите ограничение в 5 строк

In [12]:
sql = f"""
SELECT
  id,
  CASE
    WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen']) = 1 THEN if(cancellation_policy = 'flexible', 'good', 'ok')
    ELSE 'not ok'
  END AS type,
  cancellation_policy,
  amenities
FROM listings
LIMIT 5
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,id,type,cancellation_policy,amenities
0,2015,ok,strict_14_with_grace_period,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil..."
1,2695,good,flexible,"{Wifi,Kitchen,Elevator,Heating,Washer,Essentia..."
2,3176,ok,strict_14_with_grace_period,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc..."
3,3309,not ok,strict_14_with_grace_period,"{Internet,Wifi,""Pets allowed"",""Pets live on th..."
4,7071,not ok,moderate,"{Wifi,Heating,""Family/kid friendly"",Essentials..."
